## Wikipedia Scraping

In [1]:
from dotenv import load_dotenv
import os

# Load the .env file (located in the project root)
load_dotenv()

# Get the API keys and secrets
OMDB_API_KEY = os.getenv('OMDB_API_KEY')
S3_BUCKET_NAME = os.getenv('S3_BUCKET_NAME')

# Get AWS credentials
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_DEFAULT_REGION = os.getenv('AWS_DEFAULT_REGION')  

# Verify they were loaded (optional - remove in production)
if not OMDB_API_KEY:
    print("Warning: OMDB_API_KEY not found in .env file")
if not S3_BUCKET_NAME:
    print("Warning: S3_BUCKET_NAME not found in .env file")
if not AWS_ACCESS_KEY_ID or not AWS_SECRET_ACCESS_KEY:
    print("Warning: AWS credentials not found in .env file")

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_Marvel_Cinematic_Universe_films"

headers = {
    "User-Agent": "Mozilla/5.0"
}
response = requests.get(url, headers=headers)
response.status_code


200

In [4]:
response.text

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>List of Marvel Cinematic Universe films - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature

In [5]:

soup = BeautifulSoup(response.content, 'html.parser')
soup

<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of Marvel Cinematic Universe films - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-cus

In [6]:
tables =  soup.find_all('table', {'class': 'wikitable'})  #looks through the parsed HTML and finds all tables with the class 'wikitable'
tables

[<table class="wikitable plainrowheaders defaultcenter col2left" style="width: 99%;">
 <caption><style data-mw-deduplicate="TemplateStyles:r1152813436">.mw-parser-output .sr-only{border:0;clip:rect(0,0,0,0);clip-path:polygon(0px 0px,0px 0px,0px 0px);height:1px;margin:-1px;overflow:hidden;padding:0;position:absolute;width:1px;white-space:nowrap}</style><span class="sr-only">Phase One films</span>
 </caption>
 <tbody><tr>
 <th scope="col">Film<sup class="reference" id="cite_ref-DigitalSpyPhases_42-1"><a href="#cite_note-DigitalSpyPhases-42"><span class="cite-bracket">[</span>41<span class="cite-bracket">]</span></a></sup>
 </th>
 <th scope="col">U.S. release date
 </th>
 <th scope="col">Director
 </th>
 <th scope="col">Screenwriter(s)
 </th>
 <th scope="col">Producer(s)
 </th></tr>
 <tr>
 <th scope="row"><i><a href="/wiki/Iron_Man_(2008_film)" title="Iron Man (2008 film)">Iron Man</a></i>
 </th>
 <td>May 2, 2008<span style="display: none;"> (<span class="bday dtstart published updated it

In [7]:
[ele.text.strip() for ele in tables[0].find_all('tr')[1].find_all(['td', 'th'])]

['Iron Man',
 'May\xa02,\xa02008\xa0(2008-05-02)',
 'Jon Favreau[44]',
 'Mark Fergus & Hawk Ostby and Art Marcum & Matt Holloway[44][45]',
 'Avi Arad and Kevin Feige']

In [8]:
all_data = []
for i, table in enumerate(tables[:7]):  # All phases tables
    headers = [header.text.strip() for header in table.find_all('th')]
    rows = table.find_all('tr')
    data = []
    columns = ['film', 'release_date', 'director', 'writer', 'producer', "status"]
    
    for row in rows:
        cols = row.find_all(['td', 'th'])
        cols = [elem.text.strip() for elem in cols]
        if len(cols) < len(columns):
            cols.extend([None] * (len(columns) - len(cols)))
        elif len(cols) > len(columns):
            cols = cols[:len(columns)]
        data.append(cols)
    
    df = pd.DataFrame(data[1:], columns=columns)  
    df['phase'] = f"Phase {i+1}"
    all_data.append(df)

movies_df = pd.concat(all_data, ignore_index=True)

In [9]:
display(all_data)


[                                 film                release_date  \
 0                            Iron Man    May 2, 2008 (2008-05-02)   
 1                 The Incredible Hulk  June 13, 2008 (2008-06-13)   
 2                          Iron Man 2    May 7, 2010 (2010-05-07)   
 3                                Thor    May 6, 2011 (2011-05-06)   
 4  Captain America: The First Avenger  July 22, 2011 (2011-07-22)   
 5                        The Avengers    May 4, 2012 (2012-05-04)   
 
               director                                             writer  \
 0      Jon Favreau[44]  Mark Fergus & Hawk Ostby and Art Marcum & Matt...   
 1  Louis Leterrier[46]                                       Zak Penn[47]   
 2      Jon Favreau[48]                                 Justin Theroux[49]   
 3  Kenneth Branagh[50]  Ashley Edward Miller & Zack Stentz and Don Pay...   
 4     Joe Johnston[52]           Christopher Markus & Stephen McFeely[53]   
 5      Joss Whedon[54]                 

In [10]:
movies_df.head()

,film,release_date,director,writer,producer,status,phase
0,Iron Man,"May 2, 2008 (2008-05-02)",Jon Favreau[44],Mark Fergus & Hawk Ostby and Art Marcum & Matt...,Avi Arad and Kevin Feige,None,Phase 1
1,The Incredible Hulk,"June 13, 2008 (2008-06-13)",Louis Leterrier[46],Zak Penn[47],"Avi Arad, Gale Anne Hurd, and Kevin Feige",None,Phase 1
2,Iron Man 2,"May 7, 2010 (2010-05-07)",Jon Favreau[48],Justin Theroux[49],Kevin Feige,None,Phase 1
3,Thor,"May 6, 2011 (2011-05-06)",Kenneth Branagh[50],Ashley Edward Miller & Zack Stentz and Don Pay...,NaN,None,Phase 1
4,Captain America: The First Avenger,"July 22, 2011 (2011-07-22)",Joe Johnston[52],Christopher Markus & Stephen McFeely[53],NaN,None,Phase 1


In [11]:
display(movies_df)

,film,release_date,director,writer,producer,status,phase
0,Iron Man,"May 2, 2008 (2008-05-02)",Jon Favreau[44],Mark Fergus & Hawk Ostby and Art Marcum & Matt...,Avi Arad and Kevin Feige,None,Phase 1
1,The Incredible Hulk,"June 13, 2008 (2008-06-13)",Louis Leterrier[46],Zak Penn[47],"Avi Arad, Gale Anne Hurd, and Kevin Feige",None,Phase 1
2,Iron Man 2,"May 7, 2010 (2010-05-07)",Jon Favreau[48],Justin Theroux[49],Kevin Feige,None,Phase 1
3,Thor,"May 6, 2011 (2011-05-06)",Kenneth Branagh[50],Ashley Edward Miller & Zack Stentz and Don Pay...,NaN,None,Phase 1
4,Captain America: The First Avenger,"July 22, 2011 (2011-07-22)",Joe Johnston[52],Christopher Markus & Stephen McFeely[53],NaN,None,Phase 1
5,The Avengers,"May 4, 2012 (2012-05-04)",Joss Whedon[54],NaN,NaN,None,Phase 1
6,Iron Man 3,"May 3, 2013 (2013-05-03)",Shane Black[55],Drew Pearce & Shane Black[55][56],Kevin Feige,None,Phase 2
7,Thor: The Dark World,"November 8, 2013 (2013-11-08)",Alan Taylor[57],Christopher L. Yost and Christopher Markus & S...,NaN,None,Phase 2
8,Captain America: The Winter Soldier,"April 4, 2014 (2014-04-04)",Anthony and Joe Russo[59],Christopher Markus & Stephen McFeely[60],NaN,None,Phase 2
9,Guardians of the Galaxy,"August 1, 2014 (2014-08-01)",James Gunn[61],James Gunn and Nicole Perlman[62],NaN,None,Phase 2


In [12]:
movies_df['producer'] = movies_df['producer'].ffill()  #Forward fill, copies the previous
movies_df['status']   = movies_df['status'].ffill()

def remove_references(text):
    return re.sub(r'\s*\[\s*\d+\s*\]', '', text) # Removes those -> " [ 10 ]"

movies_df_cleaned = movies_df.map(
                                lambda cell: remove_references(cell) if isinstance(cell, str) else cell)  # If str, then remove references

movies_df_cleaned['release_date'] = pd.to_datetime(
    movies_df_cleaned['release_date'].str.extract(r'\((.*?)\)')[0], errors='coerce'
)
#movies_df_cleaned['release_date'] = movies_df_cleaned['release_date'].dt.strftime('%Y-%m-%d')


movies_df_cleaned

,film,release_date,director,writer,producer,status,phase
0,Iron Man,2008-05-02,Jon Favreau,Mark Fergus & Hawk Ostby and Art Marcum & Matt...,Avi Arad and Kevin Feige,NaN,Phase 1
1,The Incredible Hulk,2008-06-13,Louis Leterrier,Zak Penn,"Avi Arad, Gale Anne Hurd, and Kevin Feige",NaN,Phase 1
2,Iron Man 2,2010-05-07,Jon Favreau,Justin Theroux,Kevin Feige,NaN,Phase 1
3,Thor,2011-05-06,Kenneth Branagh,Ashley Edward Miller & Zack Stentz and Don Payne,Kevin Feige,NaN,Phase 1
4,Captain America: The First Avenger,2011-07-22,Joe Johnston,Christopher Markus & Stephen McFeely,Kevin Feige,NaN,Phase 1
5,The Avengers,2012-05-04,Joss Whedon,NaN,Kevin Feige,NaN,Phase 1
6,Iron Man 3,2013-05-03,Shane Black,Drew Pearce & Shane Black,Kevin Feige,NaN,Phase 2
7,Thor: The Dark World,2013-11-08,Alan Taylor,Christopher L. Yost and Christopher Markus & S...,Kevin Feige,NaN,Phase 2
8,Captain America: The Winter Soldier,2014-04-04,Anthony and Joe Russo,Christopher Markus & Stephen McFeely,Kevin Feige,NaN,Phase 2
9,Guardians of the Galaxy,2014-08-01,James Gunn,James Gunn and Nicole Perlman,Kevin Feige,NaN,Phase 2


In [13]:
# Status updates
movies_df_cleaned.loc[0:37, 'status'] = "Released"
movies_df_cleaned.loc[39, 'status'] = "Pre-production"
movies_df_cleaned.loc[40:, 'status'] = "In development"

# Producer updates
movies_df_cleaned.loc[[31, 32], 'producer'] = "Kevin Feige"
movies_df_cleaned.loc[40, 'producer'] = "Kevin Feige and Nate Moore"
movies_df_cleaned.loc[41, 'producer'] = "Kevin Feige"
movies_df_cleaned.loc[42, 'producer'] = "Kevin Feige"
movies_df_cleaned.loc[43, 'producer'] = "Kevin Feige and Jonathan Schwartz"  
movies_df_cleaned.loc[44, 'producer'] = "Kevin Feige"  

# Writers
movies_df_cleaned.loc[43, 'writer'] = "Destin Daniel Cretton"
movies_df_cleaned.loc[40, 'writer'] = "Ryan Coogler"
movies_df_cleaned.loc[39, 'writer'] = "Michael Waldron and Stephen McFeely"
movies_df_cleaned.loc[31, 'writer'] = "James Gunn"
movies_df_cleaned.loc[5,  'writer'] = "Joss Whedon"
movies_df_cleaned.loc[10, 'writer'] = "Joss Whedon"
movies_df_cleaned.loc[14, 'writer'] = "James Gunn"

# Directors
movies_df_cleaned.loc[39, 'director'] = "Anthony and Joe Russo"
movies_df_cleaned.loc[40, 'director'] = "Ryan Coogler"

In [14]:
movies_df_cleaned

,film,release_date,director,writer,producer,status,phase
0,Iron Man,2008-05-02,Jon Favreau,Mark Fergus & Hawk Ostby and Art Marcum & Matt...,Avi Arad and Kevin Feige,Released,Phase 1
1,The Incredible Hulk,2008-06-13,Louis Leterrier,Zak Penn,"Avi Arad, Gale Anne Hurd, and Kevin Feige",Released,Phase 1
2,Iron Man 2,2010-05-07,Jon Favreau,Justin Theroux,Kevin Feige,Released,Phase 1
3,Thor,2011-05-06,Kenneth Branagh,Ashley Edward Miller & Zack Stentz and Don Payne,Kevin Feige,Released,Phase 1
4,Captain America: The First Avenger,2011-07-22,Joe Johnston,Christopher Markus & Stephen McFeely,Kevin Feige,Released,Phase 1
5,The Avengers,2012-05-04,Joss Whedon,Joss Whedon,Kevin Feige,Released,Phase 1
6,Iron Man 3,2013-05-03,Shane Black,Drew Pearce & Shane Black,Kevin Feige,Released,Phase 2
7,Thor: The Dark World,2013-11-08,Alan Taylor,Christopher L. Yost and Christopher Markus & S...,Kevin Feige,Released,Phase 2
8,Captain America: The Winter Soldier,2014-04-04,Anthony and Joe Russo,Christopher Markus & Stephen McFeely,Kevin Feige,Released,Phase 2
9,Guardians of the Galaxy,2014-08-01,James Gunn,James Gunn and Nicole Perlman,Kevin Feige,Released,Phase 2


In [15]:
def scrape_characters_data():
    table = None
    
    for tbl in soup.find_all('table', class_='wikitable'):
        caption = tbl.find('caption')
        if caption and "Recurring cast and characters of Marvel Cinematic Universe films" in caption.get_text():  # First ensure to exist
            table = tbl
            break

    if table:
        headers = [th.get_text(strip=True) for th in table.find_all('tr')[0].find_all('th')]
        rows = []
        
        for tr in table.find_all('tr')[1:]:
            row = []
            for td in tr.find_all(['th', 'td']):
                colspan = td.get('colspan')
                if colspan:
                    colspan = int(colspan)
                    row.extend([td.get_text(separator=" ", strip=True)] * colspan)
                else:
                    row.append(td.get_text(separator=" ", strip=True))

            while len(row) < len(headers):
                row.append(None)

            if len(row) > len(headers):
                row = row[:len(headers)]  # Truncate to match header length
            
            rows.append(row)

        return pd.DataFrame(rows, columns=headers)
    else:
        print("Table with specified caption not found.")
        return pd.DataFrame()



In [16]:
characters_df = scrape_characters_data().map(lambda cell: remove_references(cell) if isinstance(cell, str) else cell)
characters_df


,Character,Phase One,Phase Two,Phase Three,Phase Four,Phase Five,Phase Six
0,Bruce Banner Hulk,Edward Norton Lou Ferrigno V Mark Ruffalo,Mark Ruffalo,Mark Ruffalo,Mark Ruffalo C,,Mark Ruffalo
1,"James ""Bucky"" Barnes Winter Soldier / White Wolf",Sebastian Stan,Sebastian Stan,Sebastian Stan,,Sebastian Stan,Sebastian Stan
2,Clint Barton Hawkeye,Jeremy Renner,Jeremy Renner,Jeremy Renner,Jeremy Renner C P V,,
3,Yelena Belova,,,,Florence Pugh,Florence Pugh,Florence Pugh
4,Peggy Carter,Hayley Atwell,Hayley Atwell,Hayley Atwell,,,
5,Carol Danvers Captain Marvel,,,Brie Larson,Brie Larson C,Brie Larson,
6,Drax the Destroyer,,Dave Bautista,Dave Bautista,Dave Bautista,Dave Bautista,
7,Jane Foster Mighty Thor,Natalie Portman,Natalie Portman,Natalie Portman,Natalie Portman,,
8,Nick Fury,Samuel L. Jackson,Samuel L. Jackson,Samuel L. Jackson,,Samuel L. Jackson,
9,Gamora,,Zoe Saldaña,Zoe Saldaña,,Zoe Saldaña,


## Omdb  API

In [17]:
def fetch_omdb_data(film_name):
    url = f'http://www.omdbapi.com/?t={film_name}&apikey={OMDB_API_KEY}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {"Title": film_name, "Error": "Data not found"}

results = []

for film in movies_df_cleaned['film']:
    movie_data = fetch_omdb_data(film)
    results.append(movie_data)


In [18]:
results_df = pd.DataFrame(results)[0:41]  # Use only indices [0:41] (inclusive)
results_df

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response,Error,totalSeasons
0,Iron Man,2008,PG-13,02 May 2008,126 min,"Action, Adventure, Sci-Fi",Jon Favreau,"Mark Fergus, Hawk Ostby, Art Marcum","Robert Downey Jr., Gwyneth Paltrow, Terrence H...","After being held captive in an Afghan cave, bi...",...,"1,205,351",tt0371746,movie,N/A,"$319,034,126",N/A,N/A,True,NaN,NaN
1,The Incredible Hulk,2008,PG-13,13 Jun 2008,112 min,"Action, Adventure, Sci-Fi",Louis Leterrier,"Zak Penn, Stan Lee, Jack Kirby","Edward Norton, Liv Tyler, Tim Roth","Bruce Banner, a scientist on the run from the ...",...,"557,033",tt0800080,movie,N/A,"$134,806,913",N/A,N/A,True,NaN,NaN
2,Iron Man 2,2010,PG-13,07 May 2010,124 min,"Action, Sci-Fi",Jon Favreau,"Justin Theroux, Stan Lee, Don Heck","Robert Downey Jr., Mickey Rourke, Gwyneth Paltrow",With the world now aware of his identity as Ir...,...,"924,662",tt1228705,movie,N/A,"$312,433,331",N/A,N/A,True,NaN,NaN
3,Thor,2011,PG-13,06 May 2011,115 min,"Action, Fantasy",Kenneth Branagh,"Ashley E. Miller, Zack Stentz, Don Payne","Chris Hemsworth, Anthony Hopkins, Natalie Portman",The powerful but arrogant god Thor is cast out...,...,"953,809",tt0800369,movie,N/A,"$181,030,624",N/A,N/A,True,NaN,NaN
4,Captain America: The First Avenger,2011,PG-13,22 Jul 2011,124 min,"Action, Adventure, Sci-Fi",Joe Johnston,"Christopher Markus, Stephen McFeely, Joe Simon","Chris Evans, Hugo Weaving, Samuel L. Jackson","Steve Rogers, a rejected military soldier, tra...",...,"958,774",tt0458339,movie,N/A,"$176,654,505",N/A,N/A,True,NaN,NaN
5,The Avengers,2012,PG-13,04 May 2012,143 min,"Action, Sci-Fi",Joss Whedon,"Joss Whedon, Zak Penn","Robert Downey Jr., Chris Evans, Scarlett Johan...",Earth's mightiest heroes must come together an...,...,"1,540,341",tt0848228,movie,N/A,"$623,357,910",N/A,N/A,True,NaN,NaN
6,Iron Man 3,2013,PG-13,03 May 2013,130 min,"Action, Adventure, Sci-Fi",Shane Black,"Drew Pearce, Shane Black, Stan Lee","Robert Downey Jr., Guy Pearce, Gwyneth Paltrow",When Tony Stark's world is torn apart by a for...,...,"957,295",tt1300854,movie,N/A,"$409,013,994",N/A,N/A,True,NaN,NaN
7,Thor: The Dark World,2013,PG-13,08 Nov 2013,112 min,"Action, Adventure, Fantasy",Alan Taylor,"Christopher L. Yost, Christopher Markus, Steph...","Chris Hemsworth, Natalie Portman, Tom Hiddleston",When the Dark Elves attempt to plunge the univ...,...,"771,012",tt1981115,movie,N/A,"$206,362,140",N/A,N/A,True,NaN,NaN
8,Captain America: The Winter Soldier,2014,PG-13,04 Apr 2014,136 min,"Action, Adventure, Sci-Fi","Anthony Russo, Joe Russo","Christopher Markus, Stephen McFeely, Joe Simon","Chris Evans, Samuel L. Jackson, Scarlett Johan...",As Steve Rogers struggles to embrace his role ...,...,"957,842",tt1843866,movie,N/A,"$259,766,572",N/A,N/A,True,NaN,NaN
9,Guardians of the Galaxy,2014,PG-13,01 Aug 2014,121 min,"Action, Adventure, Comedy",James Gunn,"James Gunn, Nicole Perlman, Dan Abnett","Chris Pratt, Vin Diesel, Bradley Cooper",A group of intergalactic criminals must pull t...,...,"1,352,030",tt2015381,movie,N/A,"$333,718,600",N/A,N/A,True,NaN,NaN


In [19]:
import boto3
from io import BytesIO
from datetime import datetime, timezone

S3_BUCKET = S3_BUCKET_NAME  
S3_PREFIX = "dim"                           # <- choose your prefix

# Create S3 client with credentials from environment variables
s3 = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_DEFAULT_REGION
)

def upload_df_as_parquet_to_s3(df: pd.DataFrame, table_name: str, bucket: str, prefix: str) -> str:
    # Optional: version your loads
    run_dt = datetime.now(timezone.utc).strftime("%Y%m%dT%H%M%SZ")

    # Convert DataFrame -> Parquet bytes in memory
    buffer = BytesIO()
    df.to_parquet(buffer, index=False, engine="pyarrow")
    buffer.seek(0)

    # S3 key (path)
    key = f"{prefix}/{table_name}/run_dt={run_dt}/part-00000.parquet"

    # Upload
    s3.put_object(Bucket=bucket, Key=key, Body=buffer.getvalue())

    return f"s3://{bucket}/{key}"

# Example usage (rename variables to match yours)
movies_path = upload_df_as_parquet_to_s3(movies_df_cleaned, "movies", S3_BUCKET, S3_PREFIX)
actors_path = upload_df_as_parquet_to_s3(characters_df, "actors", S3_BUCKET, S3_PREFIX)
ratings_path = upload_df_as_parquet_to_s3(results_df, "movies_rating", S3_BUCKET, S3_PREFIX)

print("Uploaded:")
print(movies_path)
print(actors_path)
print(ratings_path)


Uploaded:
s3://movies-data-pipeline-bucket/dim/movies/run_dt=20260217T203655Z/part-00000.parquet
s3://movies-data-pipeline-bucket/dim/actors/run_dt=20260217T203656Z/part-00000.parquet
s3://movies-data-pipeline-bucket/dim/movies_rating/run_dt=20260217T203656Z/part-00000.parquet
